In [81]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
import polars as pl
import numpy as np
from sweep.experiment import Experiment, ExperimentType
from model.data import DataModel, KFeaturesDefinition, k_features_factory
from erm.problems.problems import ProblemType
import logging
import subprocess

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

Define a datamodel

In [83]:
d = 100
vanilla_gaussian_diagonal = [(d, 1)]
x_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
θ_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
ω_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
δ_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
ν_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)

k_features_kwargs = {
    "x_diagonal": x_diagonal,
    "θ_diagonal": θ_diagonal,
    "ω_diagonal": ω_diagonal,
    "δ_diagonal": δ_diagonal,
    "ν_diagonal": ν_diagonal,
}

data_model = DataModel(
    d,
    normalize_matrices=False,
    data_model_factory=k_features_factory,
    factory_kwargs=k_features_kwargs,
)

Define an experiment

In [84]:
experiment = Experiment(
    state_evolution_repetitions=1,
    erm_repetitions=1,
    alphas=np.linspace(0,10,5),
    epsilons=np.array([0, 0.1]),
    lambdas=np.array([0.1]),
    taus=np.array([0.1]),
    d=d,
    experiment_type=ExperimentType.Sweep,
    data_model = data_model,
    test_against_epsilons=np.array([0,0.1]),
    erm_problem_type=ProblemType.Logistic,
    gamma_fair_error=0.01,
    name="vanilla-gaussian"
)
experiment_json = experiment.to_json()

In [85]:
with open("experiment.json", "w") as f:
    f.write(experiment_json)

In [86]:
command = ["pwd"]
subprocess.run(command)

/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS


CompletedProcess(args=['pwd'], returncode=0)

In [87]:
venv_python = ".venv/bin/python"
command = ["mpiexec", "-n", "5", venv_python, "sweep/run_sweep.py", "--json", experiment_json]
subprocess.run(command)

2025-02-13 21:10:40,997 - INFO - rank 3 - The MPI comm size is 5
2025-02-13 21:10:40,997 - INFO - rank 3 - This process has rank 3
2025-02-13 21:10:40,999 - INFO - rank 4 - The MPI comm size is 5
2025-02-13 21:10:40,999 - INFO - rank 4 - This process has rank 4
2025-02-13 21:10:41,014 - INFO - rank 2 - The MPI comm size is 5
2025-02-13 21:10:41,014 - INFO - rank 2 - This process has rank 2
2025-02-13 21:10:41,014 - INFO - rank 0 - The MPI comm size is 5
2025-02-13 21:10:41,014 - INFO - rank 0 - This process has rank 0
  0%|          | 0/20 [00:00<?, ?it/s]ERROR:root:'str' object has no attribute 'ρ'
Traceback (most recent call last):
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/sweep/run_sweep.py", line 73, in process_task
    task.result = run_state_evolution(task, data_model)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/sweep/run_sweep.py", line 48, in run_state_evolution
    overlaps = 

CompletedProcess(args=['mpiexec', '-n', '5', '.venv/bin/python', 'sweep/run_sweep.py', '--json', '{"name": "vanilla-gaussian", "state_evolution_repetitions": 1, "erm_repetitions": 1, "alphas": [0.0, 2.5, 5.0, 7.5, 10.0], "epsilons": [0.0, 0.1], "test_against_epsilons": [0.0, 0.1], "lambdas": [0.1], "taus": [0.1], "d": 100, "experiment_type": "Sweep", "data_model": "DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={\'x_diagonal\': KFeaturesDefinition(diagonal=[(100, 1)]), \'\\u03b8_diagonal\': KFeaturesDefinition(diagonal=[(100, 1)]), \'\\u03c9_diagonal\': KFeaturesDefinition(diagonal=[(100, 1)]), \'\\u03b4_diagonal\': KFeaturesDefinition(diagonal=[(100, 1)]), \'\\u03bd_diagonal\': KFeaturesDefinition(diagonal=[(100, 1)])}, name=\'\', description=\'\', gamma=1)", "erm_problem_type": "Logistic", "gamma_fair_error": 0.01}'], returncode=0)